In [1]:
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import requests
from nltk.tokenize import word_tokenize
import tensorflow as tf
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=72ca61996f92ceb6e6663bde9e1ef343af86b1bfe0c37748fdd49b1031e43ef5
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [4]:
import wikipedia
# Function to fetch data from Wikipedia
def get_wikipedia_data(topic, num_sentences):
    wikipedia.set_lang("en")
    try:
        content = wikipedia.page(topic).content
        sentences = content.split(". ")
        data = []
        for i in range(len(sentences) - num_sentences):
            data.append(" ".join(sentences[i:i+num_sentences]))
        return " ".join(data)
    except wikipedia.exceptions.DisambiguationError as e:
        print(e.options)
        return None
    except wikipedia.exceptions.PageError as e:
        print(e)
        return None

# Define the topics and the number of sentences in each sample
topics = ["Natural_language_processing", "Image_Processing", "Artificial_intelligence"]
num_sentences = 2

text = ""
for topic in topics:
    topic_data = get_wikipedia_data(topic, num_sentences)
    if topic_data:
        text += topic_data

In [5]:
text

'Natural language processing (NLP) is an interdisciplinary subfield of computer science and information retrieval It is primarily concerned with giving computers the ability to support and manipulate human language It is primarily concerned with giving computers the ability to support and manipulate human language It involves processing natural language datasets, such as text corpora or speech corpora, using either rule-based or probabilistic (i.e It involves processing natural language datasets, such as text corpora or speech corpora, using either rule-based or probabilistic (i.e statistical and, most recently, neural network-based) machine learning approaches statistical and, most recently, neural network-based) machine learning approaches The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances 

In [7]:
# importing the module
#import wikipedia

#text= wikipedia.summary("MachineLearning", sentences = 400)
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
#text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Remove punctuation but keep whitespace
text = re.sub(r'[^\w\s]', '', text)

# Remove numbers
text = re.sub(r'\d+', '', text)

# Remove non-English words
english_words = set(w.lower() for w in requests.get('https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt').text.split())
text = ' '.join(word for word in text.split() if word in english_words)

# Remove words with <= 3 letters
text = ' '.join(word for word in text.split() if len(word) > 3)

#########################################################################################
print('text length:', len(text))
print(text)
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

text length: 162070
language processing interdisciplinary subfield computer science information retrieval primarily concerned with giving computers ability support manipulate human language primarily concerned with giving computers ability support manipulate human language involves processing natural language datasets such text corpora speech corpora using either probabilistic involves processing natural language datasets such text corpora speech corpora using either probabilistic statistical most recently neural machine learning approaches statistical most recently neural machine learning approaches goal computer capable understanding contents documents including contextual nuances language within them goal computer capable understanding contents documents including contextual nuances language within them this natural language processing often borrows ideas from theoretical linguistics this natural language processing often borrows ideas from theoretical linguistics technology then ac

In [8]:
# length of text
words = word_tokenize(text)
print("Number of words in document: {}".format(len(words)))

Number of words in document: 20263


In [9]:
word_tokeniser = Tokenizer()
word_tokeniser.fit_on_texts([text])
encoded_words = word_tokeniser.texts_to_sequences([text])[0]

In [10]:
# check the size of the vocabulary
VOCABULARY_SIZE = len(word_tokeniser.word_index) + 1
print('Vocabulary Size: {}'.format(VOCABULARY_SIZE))

Vocabulary Size: 2887


In [11]:
sequences = []
sequences_words = []

MAX_SEQ_LENGTH = 5  # X will have five words, y will have the sixth word

for i in range(MAX_SEQ_LENGTH, len(encoded_words)):
    sequence = encoded_words[i-MAX_SEQ_LENGTH:i+1]
    sequence_words = words[i-MAX_SEQ_LENGTH:i+1]

    sequences.append(sequence)
    sequences_words.append(sequence_words)
sequences = np.array(sequences)

In [12]:
print('Total number of training samples: {}'.format(len(sequences)))
print('\nSample sequences: \n{}'.format(sequences[0:3]))

Total number of training samples: 20252

Sample sequences: 
[[  9  10 637 868  43 168]
 [ 10 637 868  43 168  62]
 [637 868  43 168  62 869]]


In [13]:
# divide the sequence into X and y
sequences = np.array(sequences)

X = sequences[:,:-1]  # assign all but last words of a sequence to X
y = sequences[:,-1]

In [14]:
X.shape, y.shape

((20252, 5), (20252,))

In [15]:
y = to_categorical(y, num_classes=VOCABULARY_SIZE)


In [16]:
y.shape

(20252, 2887)

In [17]:
X = pad_sequences(X, maxlen=MAX_SEQ_LENGTH, padding='pre')
print('Input sequence length: {}'.format(MAX_SEQ_LENGTH))

Input sequence length: 5


In [18]:
from gensim.models import FastText
fast_text_model = FastText(sentences=sequence_words,vector_size=100, window=5, min_count=1, workers=4, sg=1,epochs=500)

In [19]:
EMBEDDING_SIZE  = 100  # each word in word2vec model is represented using a 300 dimensional vector
VOCABULARY_SIZE = len(word_tokeniser.word_index) + 1

# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))

# create a word to index dictionary mapping
word2id = word_tokeniser.word_index

# copy vectors from word2vec model to the words present in corpus
for word, index in word2id.items():
      embedding_weights[index, :] = fast_text_model.wv[word]

In [20]:
embedding_weights.shape

(2887, 100)

In [21]:
model_wv = Sequential()

# embedding layer
model_wv.add(Embedding(VOCABULARY_SIZE, EMBEDDING_SIZE, input_length = MAX_SEQ_LENGTH,
                    weights = [embedding_weights], trainable=True))

# Simple RNN layer 1
model_wv.add(SimpleRNN(128, return_sequences=True))

# Simple RNN layer 2
model_wv.add(SimpleRNN(128))

# output layer
model_wv.add(Dense(VOCABULARY_SIZE, activation='softmax'))

In [22]:
# compile network
model_wv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# summarize defined model
model_wv.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            288700    
                                                                 
 simple_rnn (SimpleRNN)      (None, 5, 128)            29312     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 2887)              372423    
                                                                 
Total params: 723331 (2.76 MB)
Trainable params: 723331 (2.76 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model_wv.fit(X, y, epochs=50, verbose=1, batch_size=60)

Epoch 1/50
338/338 [==============================] - 14s 33ms/step - loss: 7.4325 - accuracy: 0.0222
Epoch 2/50
338/338 [==============================] - 6s 18ms/step - loss: 6.7188 - accuracy: 0.0290
Epoch 3/50
338/338 [==============================] - 5s 15ms/step - loss: 5.7863 - accuracy: 0.0813
Epoch 4/50
338/338 [==============================] - 5s 14ms/step - loss: 4.7410 - accuracy: 0.2141
Epoch 5/50
338/338 [==============================] - 6s 18ms/step - loss: 3.7695 - accuracy: 0.3843
Epoch 6/50
338/338 [==============================] - 5s 14ms/step - loss: 2.9215 - accuracy: 0.5397
Epoch 7/50
338/338 [==============================] - 6s 17ms/step - loss: 2.2379 - accuracy: 0.6499
Epoch 8/50
338/338 [==============================] - 5s 14ms/step - loss: 1.7245 - accuracy: 0.7361
Epoch 9/50
338/338 [==============================] - 5s 14ms/step - loss: 1.3323 - accuracy: 0.7937
Epoch 10/50
338/338 [==============================] - 6s 18ms/step - loss: 1.0467 - accur

In [28]:
import numpy as np

def generate_words(model, word_tokenizer, MAX_SEQ_LENGTH, seed, n_words):

    text = seed

    # generate n_words
    for _ in range(n_words):

        # encode text as integers
        encoded_words = word_tokenizer.texts_to_sequences([text])[0]

        # pad sequences
        padded_words = pad_sequences([encoded_words], maxlen=MAX_SEQ_LENGTH, padding='pre')

        # predict next word probabilities
        prediction = model.predict(padded_words, verbose=0)

        # convert predicted probabilities to index of the predicted word
        predicted_index = np.argmax(prediction)

        # convert predicted index to its word
        next_word = ""
        for word, i in word_tokenizer.word_index.items():
            if i == predicted_index:
                next_word = word
                break

        # append predicted word to text
        text += " " + next_word

    return text

# Example usage
seed_text = "Natural language processing (NLP) is an interdisciplinary subfield of computer science and information"
num_words = 11
print(generate_words(model_wv, word_tokeniser, MAX_SEQ_LENGTH, seed_text, num_words))


Natural language processing (NLP) is an interdisciplinary subfield of computer science and information retrieval primarily concerned with giving computers ability support manipulate human language


In [29]:
def sen_chars(text, size, step):
    text = text.replace("\n", " ").replace("\t", " ")
    chars_list = list(text)
    sp_list = [chars_list[i:i+size] for i in range(0, len(chars_list) - size + 1, step)]
    labels = [chars_list[i+size] if i+size < len(chars_list) else ' ' for i in range(0, len(chars_list) - size + 1, step)]

    return sp_list, len(sp_list) ,labels

size = 40
step = 3
chars_list, list_size,labels = sen_chars(text, size, step)

#for sp in chars_list:
#    print(sp)
len(chars_list)

54011

In [30]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 27


In [31]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
# Convert characters to integers in the sp_list and labels
sp_list_int = [[char_indices[char] for char in sp] for sp in chars_list]
labels_int = [char_indices[label] for label in labels]
sentences_encoded= np.array(sp_list_int)
labels_encoded = np.array(labels_int)

max_length = 40
X = pad_sequences(sentences_encoded, maxlen=max_length, padding='post')


In [32]:
# one hot encode outputs
from keras.utils import to_categorical
y = to_categorical(labels_encoded, num_classes=len(chars))

In [33]:
X.shape,y.shape

((54011, 40), (54011, 27))

In [34]:
'''
from gensim.models import FastText
fast_text_model = FastText(sentences=chars_list,vector_size=100, window=5, min_count=1, workers=4, sg=1,epochs=500)

EMBEDDING_SIZE = 100

# Create an empty embedding matrix
embedding_weights = np.zeros((len(chars), EMBEDDING_SIZE))

# Get embeddings for each character
for char, index in char_indices.items():
    if char in fast_text_model.wv:
        embedding_weights[index] = fast_text_model.wv[char]

# Now, embedding_weights contains embeddings for each character
print("Embedding matrix shape:", embedding_weights.shape)
'''

'\nfrom gensim.models import FastText\nfast_text_model = FastText(sentences=chars_list,vector_size=100, window=5, min_count=1, workers=4, sg=1,epochs=500)\n\nEMBEDDING_SIZE = 100\n\n# Create an empty embedding matrix\nembedding_weights = np.zeros((len(chars), EMBEDDING_SIZE))\n\n# Get embeddings for each character\nfor char, index in char_indices.items():\n    if char in fast_text_model.wv:\n        embedding_weights[index] = fast_text_model.wv[char]\n\n# Now, embedding_weights contains embeddings for each character\nprint("Embedding matrix shape:", embedding_weights.shape)\n'

In [36]:
model_cv = Sequential()
EMBEDDING_SIZE=100
# embedding layer
model_cv.add(Embedding(len(chars), EMBEDDING_SIZE, input_length = max_length))

# Simple RNN layer 1
model_cv.add(SimpleRNN(128, return_sequences=True))

# Simple RNN layer 2
model_cv.add(SimpleRNN(128))

# output layer
model_cv.add(Dense(len(chars), activation='softmax'))

In [37]:
# compile network
model_cv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# summarize defined model
model_cv.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 100)           2700      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 40, 128)           29312     
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 27)                3483      
                                                                 
Total params: 68391 (267.15 KB)
Trainable params: 68391 (267.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
model_cv.fit(X, y, epochs=10, verbose=1, batch_size=60)

Epoch 1/10
901/901 [==============================] - 71s 75ms/step - loss: 2.3760 - accuracy: 0.2845
Epoch 2/10
901/901 [==============================] - 68s 76ms/step - loss: 1.8809 - accuracy: 0.4305
Epoch 3/10
901/901 [==============================] - 67s 74ms/step - loss: 1.6334 - accuracy: 0.5023
Epoch 4/10
901/901 [==============================] - 70s 78ms/step - loss: 1.4759 - accuracy: 0.5480
Epoch 5/10
901/901 [==============================] - 68s 75ms/step - loss: 1.3682 - accuracy: 0.5773
Epoch 6/10
901/901 [==============================] - 67s 75ms/step - loss: 1.2898 - accuracy: 0.5987
Epoch 7/10
901/901 [==============================] - 69s 76ms/step - loss: 1.2258 - accuracy: 0.6176
Epoch 8/10
901/901 [==============================] - 69s 77ms/step - loss: 1.1744 - accuracy: 0.6314
Epoch 9/10
901/901 [==============================] - 68s 75ms/step - loss: 1.1291 - accuracy: 0.6446
Epoch 10/10
901/901 [==============================] - 67s 75ms/step - loss: 1.089

In [49]:
# Function to generate text
def generate_text(model, seed_text, num_chars, char_indices, indices_char):
    generated_text = seed_text
    for _ in range(num_chars):
        # Encode the seed text
        encoded_text = [char_indices[char] for char in generated_text]
        encoded_text = pad_sequences([encoded_text], maxlen=40, padding='post')
        # Predict the next character probabilities
        y_pred = model_cv.predict(encoded_text, verbose=0)[0]
        # Sample the next character based on the predicted probabilities
        next_index = np.argmax(y_pred)

       # next_index = np.random.choice(len(indices_char), p=y_pred)
        next_char = indices_char[next_index]
        # Update the generated text
        generated_text += next_char
    return generated_text

seed_text = "language processing is an interdisciplinary subfield of computer science and information"
num_chars = 35
# Generate text using the trained model
generated_text = generate_text(model_cv, seed_text, num_chars, char_indices, indices_char)

print(generated_text)


language processing is an interdisciplinary subfield of computer science and information that their artificial intelligent 
